In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sqlite3
import preprocesser
import drawing_chart
import plotly.offline as offline
import datetime

In [2]:
#시세데이터 뽑기
def query_ohlcv(shcode, fr = '20190101',to = '20190927'):
    conn = sqlite3.connect("t1305_1.db")
    cur = conn.cursor()
    query = "select 일자, 시가, 고가, 저가, 종가, 누적거래량 \
            from 종목별체결조회 \
            where 1=1 \
            and 일자 between '%s' and '%s' \
            and 종목코드 = '%s' \
            and 일주월구분 = '%s' ;" % (fr,to,shcode,'1')
    cur.execute(query)
    query_result = cur.fetchall()
    fin_result = pd.DataFrame(query_result, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    daily = fin_result.sort_values(by = ['date']).reset_index(drop = True)
    conn.close()
    return daily

In [3]:
#종목코드열람
conn = sqlite3.connect("t8430.db")
cur = conn.cursor()
query = "select distinct 종목명, 종목코드 \
        from 종목코드 ;"
        #where 종목코드 = '%s'; " % '000020'
cur.execute(query)
rows = cur.fetchall()
shcode_list = pd.DataFrame(rows, columns = ['shname', 'shcode'])
conn.close()

#1.데이터 index추출
daily = query_ohlcv(shcode_list['shcode'].iloc[0])
date = daily['date']
#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000.csv", dtype ={'shcode':str})
#3. target index 추출
fin_result = pd.DataFrame([],columns = ['interval_init','target_day','interval_end','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_index = v.split(" ")
    target_day_list = pd.to_datetime(date.iloc[target_index], format = "%Y-%m-%d")
    
    #interval calculate before 4 weeks , after 2 weeks
    interval_init_timestamp = target_day_list - datetime.timedelta(weeks=8)
    interval_start_timestamp = target_day_list - datetime.timedelta(weeks=4)
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=2)
    
    # make df using series(datetime to str)
    result = pd.DataFrame(dict(interval_init = interval_init_timestamp.dt.strftime("%Y%m%d"),
                               interval_start = interval_start_timestamp.dt.strftime("%Y%m%d"),
                               target_day = target_day_list.dt.strftime("%Y%m%d"),
                               interval_end = interval_end_timestamp.dt.strftime("%Y%m%d"))).reset_index()
    
    del result['index']
    result['shcode'] = shcode
    
    fin_result = fin_result.append(result)

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [4]:
fin_result.head()

,interval_end,interval_init,interval_start,shcode,target_day
0,20190812,20190603,20190701,000140,20190729
0,20190529,20190320,20190417,000890,20190515
0,20190327,20190116,20190213,001140,20190313
0,20191001,20190723,20190820,001340,20190917
0,20190522,20190313,20190410,001550,20190508


In [5]:
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

In [6]:
fin_result.groupby(['shcode'])['target_day'].count().sort_values(ascending = False).head()

shcode
014580    6
250060    4
044960    4
093380    4
900340    3
Name: target_day, dtype: int64

In [7]:
fin_result.groupby(['shcode','target_m']).filter(lambda x : (x['shcode'] == '014580').any())

,interval_end,interval_init,interval_start,shcode,target_day,target_m,shname
85,20190614,20190405,20190503,014580,20190531,201905,백광소재
86,20190618,20190409,20190507,014580,20190604,201906,백광소재
87,20190621,20190412,20190510,014580,20190607,201906,백광소재
88,20190823,20190614,20190712,014580,20190809,201908,백광소재
89,20191002,20190724,20190821,014580,20190918,201909,백광소재
90,20191004,20190726,20190823,014580,20190920,201909,백광소재


In [8]:
shcode_list.columns

Index(['shname', 'shcode'], dtype='object')

In [9]:
fin_result.head()

,interval_end,interval_init,interval_start,shcode,target_day,target_m,shname
0,20190812,20190603,20190701,000140,20190729,201907,하이트진로홀딩스
1,20190529,20190320,20190417,000890,20190515,201905,보해양조
2,20190327,20190116,20190213,001140,20190313,201903,카리스국보
3,20191001,20190723,20190820,001340,20190917,201909,백광산업
4,20190522,20190313,20190410,001550,20190508,201905,조비


In [10]:
fin_result['shcode'].iloc[i]

'115440'

In [11]:
fin_result.head()

,interval_end,interval_init,interval_start,shcode,target_day,target_m,shname
0,20190812,20190603,20190701,000140,20190729,201907,하이트진로홀딩스
1,20190529,20190320,20190417,000890,20190515,201905,보해양조
2,20190327,20190116,20190213,001140,20190313,201903,카리스국보
3,20191001,20190723,20190820,001340,20190917,201909,백광산업
4,20190522,20190313,20190410,001550,20190508,201905,조비


In [14]:
#데이터추출
i = 6
df = query_ohlcv(shcode = fin_result['shcode'].iloc[i],
                 fr=fin_result['interval_init'].iloc[i],
                 to=fin_result['interval_end'].iloc[i])
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
#데이터전처리
df = preprocesser.sma(df)
df = preprocesser.vma(df)
df = preprocesser.rsi(df)
df = preprocesser.macd(df)
df = preprocesser.stochastic(df)
df = preprocesser.marking_cross_sma(df)
df = preprocesser.marking_cross_rsi(df)
df = preprocesser.marking_cross_macd(df)
df = preprocesser.marking_cross_stochastic(df)
fig, fig_go = drawing_chart.drawing_chart(df)

In [15]:
print('1.종목코드: ' , fin_result['shcode'].iloc[i], '   2.종목명: ', fin_result['shname'].iloc[i], '   3.대상날짜: ', fin_result['target_day'].iloc[i])
fig_go

1.종목코드:  001745    2.종목명:  SK네트웍스우    3.대상날짜:  20190418


FigureWidget({
    'data': [{'close': array([ 57100,  56800,  57000,  56700,  56400,  55700,  55000,  56000,  …

In [ ]:
#open
#offline.iplot(fig, filename="candlestick")

#save
#offline.plot(fig, filename = 'filename.html', auto_open=False)